# Лабораторна робота №5
# Тема: Робота з бібліотекою OpenCV
# Мета: Отримання навичок роботи з бібліотекою OpenCV для обробки та аналізу цифрових зображень

<div style="background-color: white;">
Виконав: студент групи КН - 222c Максименко Валентин Михайлович 
</div>
<div style="background-color: white;">
Перевірив: асистент кафедри Герман Олександрович Зверцев
</div>

1. Визначити номер варіанта за формулою:
N = ord(X) % 5 + 1,
де X – перша велика літера вашого імені в латинській транскрипції.

In [1]:
import pandas as pd

N = ord("V") % 5 + 1 # Name: Valentine

file_path = r'C:\Users\Nikita\Desktop\Lab5_Pleshko_CS221v\lab6.xlsx'
df = pd.read_excel(file_path)

variant_data = df.iloc[N - 1]
variant_data

N                              2
file name        emma-watson.jpg
image size               400x400
glasses color               blue
line width                     3
Name: 1, dtype: object

In [3]:
print(cv2.__version__)

4.8.1


In [4]:
pip install opencv-python


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.1 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.3/38.1 MB 2.0 MB/s eta 0:00:20
    --------------------------------------- 0.5/38.1 MB 3.2 MB/s eta 0:00:12
    --------------------------------------- 0.5/38.1 MB 3.2 MB/s eta 0:00:12
   - -------------------------------------- 1.2/38.1 MB 4.9 MB/s eta 0:00:08
   - -------------------------------------- 1.4/38.1 MB 4.8 MB/s eta 0:00:08
   - -------------------------------------- 1.6/38.1 MB 4.7 MB/s eta 0:00:08
   -- ----------------

In [11]:
import cv2 as cv
import numpy as np
from PIL import Image, ImageDraw

def add_glasses_to_face(input_image_path, output_image_path, glasses_color = 'blue', line_width = 3):
    
    image = cv.imread(input_image_path)
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_eye.xml')

    detected_faces = face_cascade.detectMultiScale(gray_image, 1.3, 5)
    
    for (x, y, w, h) in detected_faces:

        roi_gray = gray_image[y:y+h, x:x+w]
        detected_eyes = eye_cascade.detectMultiScale(roi_gray)

        if len(detected_eyes) == 2:

            pil_image = Image.fromarray(cv.cvtColor(image, cv.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(pil_image)
            
            detected_eyes = sorted(detected_eyes, key=lambda e: e[0])
            
            eye1_center = (x + detected_eyes[0][0] + detected_eyes[0][2] // 2,
                           y + detected_eyes[0][1] + detected_eyes[0][3] // 2)
            eye2_center = (x + detected_eyes[1][0] + detected_eyes[1][2] // 2,
                           y + detected_eyes[1][1] + detected_eyes[1][3] // 2)

            eye_size = max(detected_eyes[0][2], detected_eyes[0][3],
                           detected_eyes[1][2], detected_eyes[1][3])
            
            distance_between_eyes = eye2_center[0] - eye1_center[0]
            glasses_width = distance_between_eyes + eye_size
            glasses_height = eye_size
            
            left_glass_rect = [eye1_center[0] - glasses_width // 4,
                               eye1_center[1] - glasses_height // 2,
                               eye1_center[0] + glasses_width // 4,
                               eye1_center[1] + glasses_height // 2]
            
            draw.ellipse(left_glass_rect, outline=glasses_color, width=line_width)
            
            right_glass_rect = [eye2_center[0] - glasses_width // 4,
                                eye2_center[1] - glasses_height // 2,
                                eye2_center[0] + glasses_width // 4,
                                eye2_center[1] + glasses_height // 2]
            
            draw.ellipse(right_glass_rect, outline=glasses_color, width=line_width)
            
            bridge_start = (eye1_center[0] + glasses_width // 4,
                            eye1_center[1])
            bridge_end = (eye2_center[0] - glasses_width // 4,
                          eye2_center[1])
            
            draw.line([bridge_start, bridge_end], fill = glasses_color, width = line_width)
            
            temple_length = 20  
            temple_left_end = (left_glass_rect[0] - temple_length, left_glass_rect[1] + glasses_height // 2)
            temple_right_end = (right_glass_rect[2] + temple_length, right_glass_rect[3] - glasses_height // 2)
            
            draw.line((left_glass_rect[0],
                       left_glass_rect[1] + glasses_height // 2) + temple_left_end, fill=glasses_color, width=line_width)
            
            draw.line((right_glass_rect[2],
                       right_glass_rect[3] - glasses_height // 2) + temple_right_end, fill=glasses_color, width=line_width)

            cv_image = cv.cvtColor(np.array(pil_image), cv.COLOR_RGB2BGR)

            face_center_x, face_center_y = x + w // 2, y + h // 2
            crop_x1 = max(face_center_x - 150, 0)
            crop_y1 = max(face_center_y - 150, 0)
            crop_x2 = min(face_center_x + 150, cv_image.shape[1])
            crop_y2 = min(face_center_y + 150, cv_image.shape[0])

            cropped_image = cv_image[crop_y1:crop_y2, crop_x1:crop_x2]
            resized_image = cv.resize(cropped_image, (400, 400), interpolation=cv.INTER_AREA)
            cv.imwrite(output_image_path, resized_image)

image_path = r"C:\Users\Nikita\Desktop\Lab5_Pleshko_CS221v\Images\ron_wesley.jpg"
output_path = r"C:\Users\Nikita\Desktop\Lab5_Pleshko_CS221v\Images\output_image.jpg"
add_glasses_to_face(image_path, output_path)

**Висновок до Лабораторної Роботи №5**

Під час виконання завдання з роботи з бібліотекою OpenCV ми успішно освоїли ключові концепції та методи обробки цифрових зображень. Використовуючи функціонал OpenCV, ми навчилися ефективно завантажувати, зберігати та обробляти зображення. Набуті знання та навички в цій області виявляться корисними у майбутній роботі, де обробка зображень використовується для вирішення різноманітних завдань, від розпізнавання об'єктів до аналізу медичних зображень та багато іншого.